# 02-01 : Use Remote Docker Instance

Attempt to use the MLRun deployed with `mlsetup.py` to run a remote docker instance.

In [1]:
import os

In [2]:
import logging

# configure log format
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s %(levelname)s [%(name)s] %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S'
)

# get the logger
logger = logging.getLogger(__name__)

## 1. Configuration 

In [ ]:
# set the api url
mlrun_api_server = 'localhost'
mlrun_api_port = 8080
os.environ["MLRUN_DBPATH"] = f"http://{mlrun_api_server}:{mlrun_api_port}"

# load the mlrun module
import mlrun

2025-05-12 11:49:35 WARNING [urllib3.connectionpool] Retrying (Retry(total=2, connect=None, read=None, redirect=None, status=None)) after connection broken by 'ProtocolError('Connection aborted.', ConnectionResetError(104, 'Connection reset by peer'))': /api/v1/client-spec
2025-05-12 11:49:37 WARNING [urllib3.connectionpool] Retrying (Retry(total=1, connect=None, read=None, redirect=None, status=None)) after connection broken by 'ProtocolError('Connection aborted.', ConnectionResetError(104, 'Connection reset by peer'))': /api/v1/client-spec
2025-05-12 11:49:41 WARNING [urllib3.connectionpool] Retrying (Retry(total=0, connect=None, read=None, redirect=None, status=None)) after connection broken by 'ProtocolError('Connection aborted.', ConnectionResetError(104, 'Connection reset by peer'))': /api/v1/client-spec


> 2025-05-12 11:49:41,418 [warning] Error during request handling, retrying: {"exc":"HTTPConnectionPool(host='localhost', port=8080): Max retries exceeded with url: /api/v1/client-spec (Caused by ProtocolError('Connection aborted.', ConnectionResetError(104, 'Connection reset by peer')))","method":"GET","retry_count":0,"url":"http://localhost:8080/api/v1/client-spec"}


2025-05-12 11:49:42 WARNING [urllib3.connectionpool] Retrying (Retry(total=2, connect=None, read=None, redirect=None, status=None)) after connection broken by 'ProtocolError('Connection aborted.', ConnectionResetError(104, 'Connection reset by peer'))': /api/v1/client-spec
2025-05-12 11:49:44 WARNING [urllib3.connectionpool] Retrying (Retry(total=1, connect=None, read=None, redirect=None, status=None)) after connection broken by 'ProtocolError('Connection aborted.', ConnectionResetError(104, 'Connection reset by peer'))': /api/v1/client-spec


In [4]:
MLRUN_PROJECT = "use-remote"
MLRUN_DOCKER_IMAGE = "mlrun/mlrun"

## 2. Set Project

In [ ]:
project = mlrun.get_or_create_project(
    name=MLRUN_PROJECT,
    context='./',) 
    
project.save()

2025-05-12 11:49:16 WARNING [urllib3.connectionpool] Retrying (Retry(total=2, connect=None, read=None, redirect=None, status=None)) after connection broken by 'NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7c203713e730>: Failed to establish a new connection: [Errno 111] Connection refused')': /api/v1/projects/use-remote
2025-05-12 11:49:18 WARNING [urllib3.connectionpool] Retrying (Retry(total=1, connect=None, read=None, redirect=None, status=None)) after connection broken by 'NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7c203713e700>: Failed to establish a new connection: [Errno 111] Connection refused')': /api/v1/projects/use-remote
2025-05-12 11:49:22 WARNING [urllib3.connectionpool] Retrying (Retry(total=0, connect=None, read=None, redirect=None, status=None)) after connection broken by 'NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7c20370f7970>: Failed to establish a new connection: [Errno 111] Connection refused')':

> 2025-05-12 11:49:22,871 [warning] Error during request handling, retrying: {"exc":"HTTPConnectionPool(host='192.168.1.184', port=8080): Max retries exceeded with url: /api/v1/projects/use-remote (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7c20370f7c70>: Failed to establish a new connection: [Errno 111] Connection refused'))","method":"GET","retry_count":0,"url":"http://192.168.1.184:8080/api/v1/projects/use-remote"}


2025-05-12 11:49:23 WARNING [urllib3.connectionpool] Retrying (Retry(total=2, connect=None, read=None, redirect=None, status=None)) after connection broken by 'NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7c20370f7d60>: Failed to establish a new connection: [Errno 111] Connection refused')': /api/v1/projects/use-remote


## 3. Create Function

### 3.1 Create Function Code File

In [ ]:
%%writefile hello_world.py

import mlrun

# get the logger
import logging
logger = logging.getLogger(__name__)

def handler(context: mlrun.MLClientCtx = None):
    logger.info("Hello world")

    context.log_result("hello", "world")

    return "Hello world result!"

### 3.2 Create Project Function

In [ ]:
hello_world = mlrun.code_to_function(
    name='hello-world',
    project=MLRUN_PROJECT,
    filename='hello_world.py',
    handler='handler',
    kind='job',
    image=MLRUN_DOCKER_IMAGE,
    description='Hello world function',
)
hello_world.save()

In [ ]:
project.build_function(hello_world)

## 4. Run Function

In [ ]:
run = hello_world.run(function=hello_world,
                      handler='handler',
                      image=MLRUN_DOCKER_IMAGE,
                      watch=True)